 <h3> Faço a correlação de pearson

In [ ]:
from functions.split_dataset import split_dataset
from functions.correlation import show_pearson_correlation

X_train, X_test, y_train, y_test = split_dataset()
show_pearson_correlation(X_train, y_train, savePng=False) # Descomentar para imprimir a correlação de pearson

 <h3> Divido o dataset com os atributos que quero

In [ ]:
import pandas as pd

# Se for usar somente alguns atributos ==========================================

#features = ['ram']
#features = ['battery_power', 'ram']
#features = ['battery_power', 'ram', 'px_height', 'px_width']

# Todos os atributos menos Ram ======================================================

#df = pd.read_csv('dataset/mobile_price_classification_dataset.csv')
#features = [col for col in df.columns if col not in ['price_range', 'ram']]

#X_train, X_test, y_train, y_test = split_dataset(selected_features = features) # Chamo novamente com os atributos que quero

# Se for com todos os atributos ==============================
X_train, X_test, y_train, y_test = split_dataset()

<h3> Fiz um scaling no dataset

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

<h3> Defino o mlp e seus parâmetros

In [ ]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(
    hidden_layer_sizes=(4), # Ficou melhor usando todos os atributos
    activation='tanh', #ficou melhor com essa
    solver='adam',
    alpha=0.001,   
    learning_rate='adaptive',
    max_iter=2000, # abaixo de 1k dá warning de não convergência
    random_state=28,
    verbose=False
)

<h1> Rodo o K-Fold para validação

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import numpy as np

# Definição do K-Fold
kf = KFold(n_splits=5, shuffle=True, random_state=28)

scores = []
for train_index, val_index in kf.split(X_train):
    X_tr, X_val = X_train[train_index], X_train[val_index]
    y_tr, y_val = y_train.iloc[train_index], y_train.iloc[val_index]

    mlp.fit(X_tr, y_tr)
    y_pred = mlp.predict(X_val)
    scores.append(accuracy_score(y_val, y_pred))

print("Acurácias em cada fold:", scores)
print("Acurácia média (validação):", np.mean(scores))

<h3> Treino o MLP

In [ ]:
mlp.fit(X_train, y_train)

y_test_pred = mlp.predict(X_test)
print(f"Acurácia no teste: {accuracy_score(y_test, y_test_pred)}")


<h3> Faz a matriz de confusão

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

cm = confusion_matrix(y_test, y_test_pred)

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=np.unique(y_test))
disp.plot(cmap=plt.cm.Blues, values_format='d', colorbar=False)  # mostra os números inteiros
plt.title("Confusion Matrix")
plt.savefig("conf_matrix", dpi=300) 
plt.show()


<h3> Calculo as métricas do modelo treinado e salvo num arquivo .txt

In [ ]:
from sklearn.metrics import classification_report
import pandas as pd

# Gera várias métricas pra avaliar o modelo
report = classification_report(y_test, y_test_pred, output_dict=True)

for key, value in report.items():
    if isinstance(value, dict): #precisa pq accuracy não é um sub-dicionario
        value.pop('support', None)

report.pop('weighted avg') # Tirei a média ponderada pq eu fiz o balanceamento das classes no conjunto de testes, então é o mesmo para todos
accuracy = report.get("accuracy")
report.pop("accuracy")

# Tenho que transformar pra dataframe pq o dict não tem o método to_string
df_report = pd.DataFrame(report).transpose() # fica mais bonito transposto
report_string = df_report.to_string()

with open("Model_metrics.txt", 'w') as f:
    f.write(report_string)
    f.write(f"\nModel accuracy: {str(accuracy)}")


<h3> Ploto e salvo num arquivo a Curva ROC (também calcula o AUC para cada classe e exibe no plot)

In [ ]:
from functions.roc_curve import calculate_roc_curve

y_pred_proba = mlp.predict_proba(X_test)
calculate_roc_curve(y_test, y_pred_proba)